# TaskTree Tutorial

In this tutorial we will walk through the capabilities of task trees in pycram.

First we have to import the necessary functionality from pycram.

In [1]:
from pycram.bullet_world import BulletWorld
from pycram.robot_descriptions.robot_description_handler import InitializedRobotDescription as robot_description
import pycram.task
from pycram.resolver.plans import Arms
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.designators.object_designator import *
import anytree
import pycram.plan_failures

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


Next we will create a bullet world with a PR2 in a kitchen containing milk and cereal.

In [2]:
world = BulletWorld()
robot = Object(robot_description.i.name, "robot", robot_description.i.name + ".urdf")
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=[1.3, 0.7, 0.95])
milk_desig = ObjectDesignator(ObjectDesignatorDescription(name="milk", type="milk"))
cereal_desig = ObjectDesignator(ObjectDesignatorDescription(name="cereal", type="cereal"))

Finally, we create a plan where the robot parks his arms, walks to the kitchen counter and picks the thingy. Then we execute the plan.

In [3]:
@pycram.task.with_tree
def plan():
    with simulated_robot:
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

        location = LocationDesignator(CostmapLocation(target=milk, reachable_for=robot))
        pose = location.reference()
        ActionDesignator(
            NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

        picked_up_arm = pose["arms"][0]
        ActionDesignator(PickUpAction(object_designator=milk_desig, arm=pose["arms"][0], grasp="front")).perform()

        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()
        place_island = LocationDesignator(SemanticCostmapLocation("kitchen_island_surface", kitchen, milk_desig.prop_value("object")))
        pose_island = place_island.reference()

        place_location = LocationDesignator(CostmapLocation(target=list(pose_island.values()), reachable_for=robot, reachable_arm=picked_up_arm))
        pose = place_location.reference()

        ActionDesignator(NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()

        ActionDesignator(PlaceAction(object_designator=milk_desig, target_location=list(pose_island.values()), arm=picked_up_arm)).perform()

        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

plan()


Parking arms Arms.BOTH.
Moving to [0.7199999690055847, 1.0399999618530273, 0.0]. Orientation: [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606].
(0.7199999690055847, 1.0399999618530273, 0.0)
Parking arms Arms.BOTH.
Picking up ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': None, 'pose': None}) with left.
Not attached to anything!
Parking arms Arms.BOTH.
Moving to [-1.9075000286102295, 0.7792000770568848, 0.0]. Orientation: [0.0, 0.0, 0.16439898301071468, 0.9863939245479175].
(-1.9075000286102295, 0.7792000770568848, 0.0)
Placing ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left at [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339889056055381, 0.7733421413379024]].
Parking arms Arms.BOTH.


Now we get the task tree from its module and render it. Rendering can be done with any render method described in the anytree package. We will use ascii rendering here for ease of displaying.

In [4]:
tt = pycram.task.task_tree
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

no_operation()
+-- plan()
    |-- park_arms(arms = Arms.BOTH)
    |-- navigate(target = [0.7199999690055847, 1.0399999618530273, 0.0], orientation = [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606])
    |-- park_arms(arms = Arms.BOTH)
    |-- pick_up(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), grasp = front)
    |-- park_arms(arms = Arms.BOTH)
    |-- navigate(target = [-1.9075000286102295, 0.7792000770568848, 0.0], orientation = [0.0, 0.0, 0.16439898301071468, 0.9863939245479175])
    |-- place(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), target = [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339889056055381,

As we see every task in the plan got recorded correctly. It is noticeable that the tree begins with a NoOperation node. This is done because several, not connected, plans that get executed after each other should still appear in the task tree. Hence, a NoOperation node is the root of any tree. If we re-execute the plan we would see them appear in the same tree even though they are not connected.

In [5]:
world.reset_bullet_world()
plan()
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

Parking arms Arms.BOTH.
Moving to [0.7199999690055847, 1.0399999618530273, 0.0]. Orientation: [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606].
(0.7199999690055847, 1.0399999618530273, 0.0)
Parking arms Arms.BOTH.
Picking up ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left.
Not attached to anything!
Parking arms Arms.BOTH.
Moving to [-1.9075000286102295, 0.7792000770568848, 0.0]. Orientation: [0.0, 0.0, 0.16439898301071468, 0.9863939245479175].
(-1.9075000286102295, 0.7792000770568848, 0.0)
Placing ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left at [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339889056055381, 0.7733421413379024]].
Parking arms Arms.BOT

Projecting a plan in a new environment with its own task tree that only exists while the projected plan is running can be done with the ``with`` keyword. When this is done, both the bullet world and task tree are saved and new, freshly reset objects are available. At the end of a with block the old state is restored. The root for such things is then called ``simulation()``.

In [6]:
with pycram.task.SimulatedTaskTree() as stt:
    print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

simulation()
no_operation()
|-- plan()
|   |-- park_arms(arms = Arms.BOTH)
|   |-- navigate(target = [0.7199999690055847, 1.0399999618530273, 0.0], orientation = [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606])
|   |-- park_arms(arms = Arms.BOTH)
|   |-- pick_up(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), grasp = front)
|   |-- park_arms(arms = Arms.BOTH)
|   |-- navigate(target = [-1.9075000286102295, 0.7792000770568848, 0.0], orientation = [0.0, 0.0, 0.16439898301071468, 0.9863939245479175])
|   |-- place(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), target = [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339

Task tree can be manipulated with ordinary anytree manipulation. If we for example want to discard the second plan, we would write

In [7]:
tt.root.children = (tt.root.children[0],)
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

no_operation()
+-- plan()
    |-- park_arms(arms = Arms.BOTH)
    |-- navigate(target = [0.7199999690055847, 1.0399999618530273, 0.0], orientation = [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606])
    |-- park_arms(arms = Arms.BOTH)
    |-- pick_up(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), grasp = front)
    |-- park_arms(arms = Arms.BOTH)
    |-- navigate(target = [-1.9075000286102295, 0.7792000770568848, 0.0], orientation = [0.0, 0.0, 0.16439898301071468, 0.9863939245479175])
    |-- place(arm = left, object_desig = ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}), target = [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339889056055381,

We can now re-execute this (modified) plan by executing the leaf in pre-ordering iteration using the anytree functionality. This will not append the re-execution to the task tree.

In [8]:
world.reset_bullet_world()
with simulated_robot:
    [node.code.execute() for node in tt.root.leaves]
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

Parking arms Arms.BOTH.
Moving to [0.7199999690055847, 1.0399999618530273, 0.0]. Orientation: [-0.0, 0.0, 0.03442140918758331, -0.9994074077119606].
(0.7199999690055847, 1.0399999618530273, 0.0)
Parking arms Arms.BOTH.
Picking up ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left.
Not attached to anything!
Parking arms Arms.BOTH.
Moving to [-1.9075000286102295, 0.7792000770568848, 0.0]. Orientation: [0.0, 0.0, 0.16439898301071468, 0.9863939245479175].
(-1.9075000286102295, 0.7792000770568848, 0.0)
Placing ObjectDesignator({'resolver': 'grounding', 'type': 'milk', 'name': 'milk', 'object': <pycram.bullet_world.Object object at 0x7f0a8346b460>, 'pose': ((1.3, 1.0, 0.9), (0.0, 0.0, 0.0, 1.0))}) with left at [[-1.2074999809265137, 1.019200086593628, 0.9439613818399623], [0.0, 0.0, 0.6339889056055381, 0.7733421413379024]].
Parking arms Arms.BOT

Nodes in the task tree contain additional information about the status and time of a task.

In [9]:
print(pycram.task.task_tree.children[0])

Code: plan() 
 start_time: 2023-04-13 11:14:37.899141 
 Status: TaskStatus.SUCCEEDED 
 end_time: 2023-04-13 11:14:42.710393 
 


The task tree can also be reset to an empty one by invoking

In [10]:
pycram.task.reset_tree()
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))

no_operation()


If a plan fails using the PlanFailure exception, the plan will not stop. Instead, the error will be logged and saved in the task tree as a failed subtask. First let's create a simple failing plan and execute it.

In [11]:
@pycram.task.with_tree
def failing_plan():
    raise pycram.plan_failures.PlanFailure("Oopsie!")

failing_plan()

We can now investigate the nodes of the tree, and we will see that the tree indeed contains a failed task.

In [12]:
print(anytree.RenderTree(pycram.task.task_tree, style=anytree.render.AsciiStyle()))
print(pycram.task.task_tree.children[0])

no_operation()
+-- failing_plan()
Code: failing_plan() 
 start_time: 2023-04-13 11:16:19.802825 
 Status: TaskStatus.FAILED 
 end_time: 2023-04-13 11:16:19.803078 
 


In [13]:
world.exit()

shutdown request: [/pycram] Reason: new node registered with same name
